# Análise dos Tópicos 

Esse notebook possui os códigos para a análise dos tópicos extraídos dos diários oficiais. 

## Imports Necessários

In [1]:
# Código para poder usar os módulos implementados nesse repositório no notebook 
import sys, os
path_module = os.path.abspath('../modules/')
if path_module not in sys.path:
    sys.path.append(path_module)

In [2]:
# imports do projeto

from utils.process_gazette import ProcessGazette
from preprocess.pre_process_text import PreProcessText
from nlp.extract_topicsLLM import ExtractTopicsLLM
from nlp.ner import Ner
from nlp.sentiment import Sentiment_analyze

## Extração e análise dos tópicos do Diário Teste

### teste de filtro

In [3]:
pp = ProcessGazette(BASE_DIR="gazettes/")
txt_files = [f for f in os.listdir("gazettes/") if f.endswith('.txt')]
all_pages = []
for txt_file in txt_files:
    pages = pp.break_pages(txt_file, "ANO [X|V|I]+ ", save_file=True)
    all_pages.append(pages)

In [4]:
#classificação
main_prompt ="""Você é um assistente de classificação especializado em diários oficiais. Dado o texto a seguir de uma página de um diário oficial, classifique-o em uma das seguintes categorias:

Licitações
Contratos
Leis
Decretos
Nomeações e Exonerações
Atos Administrativos
Publicações Judiciais
Publicações de Empresas
Editais Diversos
Avisos e Comunicados de Interesse Público
Publicações de Organismos Internacionais
Matérias de Transparência
Siga estas etapas:

Leia o texto fornecido cuidadosamente.
Classifique o texto em uma das categorias listadas.
Apresente apenas a categoria de classificação como saída, sem qualquer outro texto ou explicação.
Texto:"""

In [5]:
#resumo
main_prompt2 ="""Você é um assistente de resumo especializado em diários oficiais. Sua tarefa é ler o texto fornecido de uma página de um diário oficial e fornecer um resumo conciso que capture os principais pontos e informações relevantes.

Siga estas etapas:
Leia o texto fornecido cuidadosamente.
Identifique os principais pontos e informações relevantes.
Escreva um resumo conciso que capture esses pontos principais.
Texto:"""

In [6]:
#licitações
main_prompt3="""Você é um assistente especializado em análise de diários oficiais. Sua tarefa é extrair informações de licitações do texto fornecido. Para cada licitação, identifique a empresa, o valor e os objetivos. Apresente suas respostas em formato de dicionário Python, onde cada licitação é representada por um dicionário com as chaves "Empresa", "Valor" e "Objetivos".

Siga estas etapas:
1. Leia o texto fornecido cuidadosamente.
2. Identifique cada licitação mencionada no texto.
3. Para cada licitação, extraia as informações de "Empresa", "Valor" e "Objetivos".
4. Apresente os resultados em um dicionário Python, onde cada licitação é representada por um dicionário com as chaves "Empresa", "Valor" e "Objetivos".

Formato do dicionário:

```python
{
    "Licitação 1": {
        "Empresa": "Nome da Empresa",
        "Valor": "Valor da Licitação",
        "Objetivos": "Objetivos da Licitação"
    },
    "Licitação 2": {
        "Empresa": "Nome da Empresa",
        "Valor": "Valor da Licitação",
        "Objetivos": "Objetivos da Licitação"
    },
    ...
}
```
Apresente apenas o dicionário python como saída, sem qualquer outro texto ou explicação.
Texto:"""

In [7]:
revistas_filtradas = {}
for gazette in range(1,2):
    revistas_filtradas[gazette] = {}
    for page, text in all_pages[gazette].items():
        import maritalk
        model = maritalk.MariTalk(
        key="104147566582134244375$db094baa6042418d",
        model="sabia-2-small"  #modelos sabia-2-medium e sabia-2-small
        )
        prompt = main_prompt + str(text)
        response  = model.generate(prompt)
        answer = response["answer"]
        print(f"page:{page} {answer}")
        #if answer.strip() == "Licitações":
        revistas_filtradas[gazette][page] = {'text': text}
        revistas_filtradas[gazette][page]['classification'] = answer

page:0  Atos Administrativos
page:1  Decretos
page:2  Contratos
page:3  Nomeações e Exonerações
page:4  Licitações
page:5   Publicações de Organismos Internacionais
page:6   Decretos
page:7  Sistema Estruturado  para a Educação  Infantil - Manutenção  Exclusivo  - 1.138.000,00 1.138.000,00 - -  720.000,00 720.000,00 - -  2.400.000,00 2.400.000,00 100,00  - 5.600.000,00 5.600.000,00 100,00  - 5.600.000,00 5.600.000,00 100,00  - 1.600.000,00 1.600.000,00 100,00  - 1.600.000,00 1.600.000,00 100,00  - 3.200.000,00 3.200.000,00 100,00  - 3.200.000,00 3.200.000,00 100,00  - 800.000,00 800.000,00 100,00  - 800.000,00 800.000,00 100,00  - 2.000.000,00 2.000.000,00 100,00  - 2.000.000,00 2.000.000,00 100,00  - 2.000.000,00 2.000.000,00 100,00  - 2.000.000,00 2.000
page:8  Licitação.
page:9  300.000,00 0,0  243 – Assistência à  Criança e ao Adolescente Projeto  Manutenção dos Centros de Convivência   - FCM Não Exclusivo 1.627.000,00 626.594,95  TOTAL GERAL 3.330.000,00 0,0
page:10  Contratos
pag

In [9]:
import maritalk
model = maritalk.MariTalk(
key="104147566582134244375$db094baa6042418d",
model="sabia-2-small"  #modelos sabia-2-medium e sabia-2-small
)
prompt = main_prompt3 + revistas_filtradas[1][14]['text']
response  = model.generate(prompt)
answer = response["answer"]
print(answer)

 ```python
{
    "Licitação 1": {
        "Empresa": "MEDLEVENSOHN COMÉRCIO E REP. DE PRODUTOS HOSP. LTDA",
        "Valor": "R$ 4.750.000,00",
        "Objetivos": "Aquisição de kit para laboratório teste rápido IGM/IGG para coronavirus"
    },
    "Licitação 2": {
        "Empresa": "VIGILANTE DA GLICOSE COM DE PROD PARA DIABETICOS",
        "Valor": "R$ 50.960,00",
        "Objetivos": "REGISTRO DE PREÇOS PARA AQUISIÇÃO DE MATERIAL DE PENSO"
    },
    "Licitação 3": {
        "Empresa": "KOLPLAST CI S.A.",
        "Valor": "R$ 441.120,00",
        "Objetivos": "REGISTRO DE PREÇOS PARA AQUISIÇÃO DE MATERIAL DE PENSO"
    },
    "Licitação 4": {
        "Empresa": "CORDEIRO CARAPIA COMERCIO DE PRODUTOS HOSPITALARES",
        "Valor": "R$ 61.470,00",
        "Objetivos": "REGISTRO DE PREÇOS PARA AQUISIÇÃO DE MATERIAL DE PENSO"
    },
    "Licitação 5": {
        "Empresa": "CREMER S.A.",
        "Valor": "R$ 287.325,00",
        "Objetivos": "REGISTRO DE PREÇOS PARA AQUISIÇÃO DE MATER

In [10]:
import maritalk
model = maritalk.MariTalk(
key="104147566582134244375$db094baa6042418d",
model="sabia-2-small"  #modelos sabia-2-medium e sabia-2-small
)
prompt = main_prompt2 + revistas_filtradas[1][14]['text']
response  = model.generate(prompt)
answer = response["answer"]
print(answer)

 Resumo:

1. Dispensa de Licitação N° 050/2020 para aquisição de kits de teste rápido para coronavírus (IGM/IGG).
2. Contratada: Medlevensohn Comércio e Representações de Produtos Hospitalares Ltda.
3. CNPJ da Contratada: 05.343.029/0001-90.
4. Valor Global do Contrato: R$ 4.750.000,00.
5. Ato de Dispensa de Licitação assinado em 26/03/2020 pela Coordenadora Paloma Mendes Mendonça.
6. Amparo Legal: Artigo 24, Inciso IV da Lei Federal N° 8.666/93 e Artigo 4° da Lei Federal N° 13.979/2020.
7. Dotação Orçamentária: Projeto Atividade 10.302.0002.232900, 10.301.0016.249300 e 10.302.0016.249400; Elemento de Despesa 33.90.30; Fonte de Recurso: 014.
8. Prazo de Entrega: Imediato.
9. Resultado de Licitação: Pregão Eletrônico - SMS N° 027/2020 para aquisição de material de penso.
10. Empresas contempladas e valores por lote: Vigilante da Glicose Comércio de Produtos para Diabéticos, Kolplast CI S.A., Cordeiro Carapiá Comércio de Produtos Hospitalares, Cremer S.A. e Cremmer S.A.
11. Valor total d

In [11]:
extractor = ExtractTopicsLLM()
ner = Ner()
topics = extractor.extract_topics(revistas_filtradas[1][14]['text'])
Ner_mari = ner.extract_entities_maritalk(revistas_filtradas[1][14]['text'])

c:\Users\erick\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
print(topics)

  ```python
    {
        "Licitação": {
            "Description": "Processos de contratação e aquisição de bens e serviços pelo governo.",
            "Frequency": 3
        },
        "Dispensa de Licitação": {
            "Description": "Procedimento para contratação direta sem licitação seguindo condições específicas.",
            "Frequency": 2
        },
        "Valor Global": {
            "Description": "Valor total do contrato resultante da dispensa de licitação.",
            "Frequency": 2
        },
        "Empresa": {
            "Description": "Nome da empresa contratada por dispensa de licitação.",
            "Frequency": 2
        },
        "CNPJ": {
            "Description": "Número de identificação da empresa contratada.",
            "Frequency": 2
        },
        "Artigo": {
            "Description": "Referência legal para a dispensa de licitação.",
            "Frequency": 1
        },
        "Resultado de Licitação": {
            "Description": "Resul

In [13]:
print(Ner_mari)

  ```python
{
    "Person": ["Paloma Mendes Mendonça", "José Egídio de Santana"],
    "Organization": ["SMS - Secretaria Municipal da Saúde", "COPEL - Comissão Setorial Permanente de Licitação", "PMH Produtos Médicos Hospitalares LTDA", "Guarda Civil Municipal - GCM"],
    "Location": ["Salvador", "Salvador", "Salvador", "Salvador"],
    "Date": ["30/03/2020", "27/03/2020", "26/03/2020", "26/03/2020"],
    "Money": ["R$ 4.750.000,00", "R$ 967.625,00", "R$ 4.925.000,00", "R$ 1.000 (mil) mililitros"]
}
```


In [14]:
x = Sentiment_analyze(model='spacy')
y = Sentiment_analyze(model='maritalk')
spacy = x.sentiment_analyze_spacy(revistas_filtradas[0][2]['text'])
mari = y.sentiment_analyze_maritalk(revistas_filtradas[0][2]['text'])

KeyError: 0

In [ ]:
print(spacy)
print(mari)

positive
  Positivo


### Extraindo de multiplas gazetas

In [4]:
pp = ProcessGazette(BASE_DIR="gazettes/")
txt_files = [f for f in os.listdir("gazettes/") if f.endswith('.txt')]
all_pages = []
for txt_file in txt_files:
    pages = pp.break_pages(txt_file, "ANO [X|V|I]+ ", save_file=True)
    all_pages.append(pages)

In [5]:
cont = 1 #len(revistas_filtradas)

In [6]:
all_topics = {}
topics_ = {}
ppt = PreProcessText("pt_core_news_lg")
for gazette in range(0,cont):
    all_topics[gazette] = {}
    for page, text in all_pages[gazette].items():
        extractor = ExtractTopicsLLM()
        topics_[page] = {'text': text, 'topics': []}
        tokens_ = ppt.process_text(topics_[page]['text'])
        if len(tokens_) > 5 and page:
            topics_[page]['tokens'] = tokens_
            topics_[page]['topics'] = extractor.extract_topics(topics_[page]['text'])
            
            all_topics[gazette][page] = topics_[page]


c:\Users\erick\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### run

In [7]:
gazeta = 0
pg = 4

In [8]:
print(all_topics[gazeta][pg]['Ner_text'])

[(Capítulo, 'MISC'), (Município de Salvador, 'LOC'), (Decreto, 'MISC'), (Lei, 'MISC'), (CAPÍTULO V DO COMITÊ DE INTEGRAÇÃO INSTITUCIONAL  Art. 15, 'MISC'), (Comitê de Integração Institucional, 'ORG'), (Município de Salvador, 'LOC'), (Covid-19, 'LOC'), (Comitê de Integração Institucional, 'ORG'), (I - Prefeito Municipal, 'MISC'), (II - Vice-Prefeito, 'MISC'), (Casa Civil, 'PER'), (IV - Chefe de Gabinete do Prefeito, 'MISC'), (V - Procurador-Geral do Município, 'MISC'), (Secretário Municipal da Saúde, 'PER'), (Secretário Municipal da Fazenda, 'LOC'), (Secretário Municipal de Promoção Social e Combate à Pobreza, 'PER'), (Secretário Municipal de Educação, 'PER'), (Secretário Municipal de Desenvolvimento e Urbanismo, 'LOC'), (XIV- Secretário Municipal de Ordem Pública, 'PER'), (Controlador-Geral do Município, 'LOC'), (Comitê de Integração Institucional, 'ORG'), (Presidente, 'MISC'), (Secretários Municipais, 'PER'), (Administração Indireta Municipal, 'ORG'), (Prefeito, 'ORG'), (Vice-presiden

In [12]:
print(all_topics[gazeta][pg]['Ner_mari_txt'])

  ```python
{
    "Organization": ["Prefeito Municipal", "Vice-Prefeito", "Chefe da Casa Civil", "Chefe de Gabinete do Prefeito", "Procurador-Geral do Município", "Secretário Municipal da Saúde", "Secretário Municipal da Fazenda", "Secretário Municipal de Promoção Social e Combate à Pobreza", "Secretário Municipal de Educação", "Secretário Municipal de Infraestrutura", "Secretário Municipal de Gestão", "Secretário Municipal de Mobilidade", "Secretário Municipal de Desenvolvimento e Urbanismo", "Secretário Municipal de Ordem Pública", "Secretário Municipal de Comunicação", "Secretário-Geral de Articulação Comunitária e Prefeituras-Bairro", "Controlador-Geral do Município"],
    "Location": ["Salvador"],
    "Date": ["19 de março de 2020", "3 (três) meses"]
}
```


In [10]:
print(all_topics[gazeta][pg]['topics'])

[]
